## Overview Structure
1. Files Watchdog ( async trigger->2 )
2. Text & Info extraction (tika) ->
3. Do embedding & storage (gpt-API + chroma) ->
4. Function : Vector Search (gpt-API, Bart)
5. Function : Summarization  (gpt-API, Bart)

### 3. Chroma : Vector Store

In [1]:
# !pip install -U chromadb==0.5.0  # 0.5.4 windows crash 
import chromadb
from tqdm import tqdm
import numpy as np
from sentence_transformers import SentenceTransformer # from sentence transformer
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction # from chromdb utils
import os
from src import search_chroma, query_openai, io_local

if 0:
    em_model = SentenceTransformer('paraphrase-albert-small-v2')
    def emb_fn(texts):
        return em_model.encode(texts, convert_to_tensor=True).tolist()
else:
    emb_fn = SentenceTransformerEmbeddingFunction(model_name='paraphrase-multilingual-MiniLM-L12-v2') # HuggingFace, for Chinese

client = chromadb.PersistentClient(path="../../Data/chromabase")
# client.delete_collection(name="project-files")
collection = client.get_or_create_collection("project-files", embedding_function=emb_fn) # collection is in memory

C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
a=collection.get(include=['metadatas'])

In [7]:
import pandas as pd
b=pd.DataFrame(a['metadatas'])
b['ids'] = a['ids']

In [11]:
pd.DataFrame(io_local.list_dir_metas("E:\\数智项目资料\\NQI\\样本资产管理平台\\"))

,path,type,size,date
0,E:\数智项目资料\NQI\样本资产管理平台\一期 - 人工智能样本资源管理及可视化.docx,docx,50983,1.727059e+09
1,E:\数智项目资料\NQI\样本资产管理平台\二期 - 人工智能样本资源管理及可视化二期.docx,docx,524815,1.727059e+09


#### 3.1 Synchronize & Update local files

In [2]:
f_metas_update, db_delete = search_chroma.sync_dir(collection, 'E:\\数智项目资料\\NQI')
print('All:', collection.count(), ' Update:', len(f_metas_update), ' Delete:', len(db_delete))

collection.query(
    query_texts="如何防止电力系统火灾的发生？", include = ['distances'], n_results = 3
)

All: 32  Update: 0  Delete: 0


{'ids': [['E:\\数智项目资料\\NQI\\资料搜集\\2023申国内外现状(抽取).pdf',
   'E:\\数智项目资料\\NQI\\资料搜集\\2023年火灾防控-申报书.pdf',
   'E:\\数智项目资料\\NQI\\综述\\综述文献\\新型电力系统中人工智能应用的关键技术-蒲天骄.pdf']],
 'distances': [[17.384515741805547, 18.840491924595145, 19.725080687160215]],
 'metadatas': None,
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None}

### 4. Answer question by context

In [3]:
Q = "人工智能与电力行业结合有哪些难点?"

search_results = collection.query(
    query_texts=Q, include = ['distances','documents'], n_results = 3
)
print('searched files id: ', search_results['ids']) #  metadatas / distances / documents  
response = query_openai.get_response_openai(Q, ''.join(search_results['documents'][0]), 'gpt-4o-mini', task='refer')
response

searched files id:  [['E:\\数智项目资料\\NQI\\深圳局\\NQI-5.1.docx', 'E:\\数智项目资料\\NQI\\深圳局\\NQI-5.1_v1.docx', 'E:\\数智项目资料\\NQI\\综述\\综述文献\\数据与知识联合驱动的人工智能方法在电力调度中的应用综述-李鹏.pdf']]


'人工智能与电力行业结合的难点主要包括以下几个方面：\n\n1. **数据集构建与管理**：需要针对电力行业的复杂场景（如“源网荷储”）制定标准化的数据集，确保数据的类别、精度和数量满足行业要求，并保证数据的安全性和合规性。\n\n2. **模型验证与认证**：人工智能模型必须经过严格的检验和认证，确保其在实际应用中的准确性、适用性和稳定性，这对于电力行业来说尤其重要，因为它关系到系统的安全与稳定。\n\n3. **多模型协同与升级**：电力系统中存在多种人工智能模型，需要建立统一的模型管理体系，支持云边端模型的协同与远程升级，以适应系统的实时需求。\n\n4. **安全性与实时性**：电力行业对数据处理和模型运用的安全性和实时性要求极高，任何延迟或不准确都可能导致灾难性后果。\n\n5. **标准化体系的缺失**：目前针对电力行业的人工智能应用标准和测量指标仍然不足，缺乏具体的行业特性和定量评估标准，使得实际应用中的验证和评估变得困难。\n\n根据文档中的信息，以上这些难点体现了人工智能在电力行业实际应用中的复杂性与挑战性。'

In [110]:
### Q1: by Q_Text (Embedding Indexing)
Q = dataset['question'][:1]
results = collection.query(query_texts=Q, n_results=2, include = ['documents', 'metadatas'])

### Q2: by Q_Text + (Embedding Indexing)
# Q = dataset['question'][:5]
# results = collection.query(query_texts=Q, n_results=2, include = ['metadatas', 'documents'],
#                            where_document = {"$contains": "Chemistry"})

### Q3: by Hard filter
# results = collection.get(ids = ["11"], include = ['embeddings', 'documents'])
# results = collection.get(where = {'source':'notion'} )
results = collection.query(query_texts=Q[0], n_results=3)


In [ ]:
from openai import OpenAI
client = OpenAI()

MAIN_CFG = {
    'model':"gpt-4o-mini", # can be 'gpt-4o', 'o1-mini'        
    'messages':[
        {"role": "user", "content": "explain in 300 words what's fine-tune in llm?"}
    ],
    'max_tokens':400,
    'temperature':1
}

### Test & Debug

#### Test 'search + query' via test-doc

In [ ]:
from src import search_chroma, query_openai

q = "are there anything about sport?"
search_results = collection.query(query_texts=Q, n_results=2, include = ['documents', 'metadatas'])
print('searched files id: ', search_results['ids'][0]) #  metadatas / distances / documents  
response = query_openai.get_response_openai(q, ''.join(search_results['documents'][0]), 'gpt-4o-mini')
response

#### Test From HuggingFace dataset 

In [1]:
## large dataset : SciQ dataset from HuggingFace
from datasets import load_dataset
dataset = load_dataset("sciq", split="train")
dataset = dataset.filter(lambda x: x["support"] != "")
dataset = dataset.select(range(1000))

print("Number of questions with support: ", len(dataset))

## Load the supporting evidence in batches of 1000
col2 = client.get_or_create_collection("sciq_supports")
batch_size = 20
for i in tqdm(range(0, len(dataset), batch_size)):
    collection.add(
        ids=[ str(i) for i in range(i, min(i + batch_size, len(dataset))) ],  # IDs are just strings
        documents=dataset["support"][i : i + batch_size],
        metadatas=[
            {"type": "support"} for _ in range(i, min(i + batch_size, len(dataset)))
        ],
    )

#### Problem: Chinese embedding
- sentence embedding: 'paraphrase-multilingual-MiniLM-L12-v2'

#### Problem: Tokens too large after semantic search
- Truncate + Concatenate: File=chunks / Sums[]=summerize(chunks[]) / RC=summerize(Sums[])
- Build section/keyword-metadata + Filter by metadata
    - chunk = ["'section:'...chunk"], meta = {keywords}
    - semantic_search(chunks) + keywords_filter(keywords)
- Todo: Put file-name to metadata, multiple-trunks = 1-fname
    - ids = 'path' + str(chunk-id)
    - update: delete db_meata['path'] == file['path'] / insert
    - delete: delete db_meata['path'] == file['path']
